## **Extract key phrases**

In [1]:
ALTER TABLE listings ADD COLUMN key_phrases text[];

UPDATE listings
SET key_phrases = azure_cognitive.extract_key_phrases(description)
FROM (SELECT id FROM listings WHERE key_phrases IS NULL ORDER BY id LIMIT 100) subset
WHERE listings.id = subset.id;

SELECT id, name FROM listings WHERE 'closet' = ANY(key_phrases);

ALTER TABLE

UPDATE 50

(6 row(s) affected)

Total execution time: 00:00:08.686

id,name
3,Open Airy Condo
11,Ballard private room
17,Private bed/bath
38,Cozy 1BD
39,2 Private Rooms
5,One bedroom with Lounge


## **Named entity recognition**

In [11]:
SELECT azure_cognitive.recognize_entities('Microsoft released Azure Cognitive Services in Seattle in 2021.', 'en-us')

(1 row(s) affected)

Total execution time: 00:00:00.314

recognize_entities
"{""(Microsoft,Organization,\""\"",1)"",""(\""Azure Cognitive Services\"",Product,ComputingProduct,1)"",""(Seattle,Location,City,1)"",""(2021,DateTime,DateRange,1)""}"


In [15]:
ALTER TABLE listings ADD COLUMN entities azure_cognitive.entity[];

UPDATE listings
SET entities = azure_cognitive.recognize_entities(description, 'en-us') 
FROM (SELECT id FROM listings WHERE entities IS NOT NULL ORDER BY id LIMIT 500) as subset
WHERE listings.id = subset.id;

SELECT id, name
FROM listings, unnest(listings.entities) AS e
WHERE e.text LIKE '%basements%'
LIMIT 10;

UPDATE 50

(1 row(s) affected)

Total execution time: 00:00:08.081

id,name
5,One bedroom with Lounge


In [17]:
SELECT id, name FROM listings WHERE 'closet' = ANY(key_phrases);

(6 row(s) affected)

Total execution time: 00:00:00.248

id,name
3,Open Airy Condo
17,Private bed/bath
38,Cozy 1BD
11,Ballard private room
39,2 Private Rooms
5,One bedroom with Lounge


## PII Recognition

In [1]:
SELECT azure_cognitive.recognize_pii_entities('For more information, see Cognitive Services Compliance and Privacy notes.', 'en-us');

(1 row(s) affected)

Total execution time: 00:00:00.338

recognize_pii_entities
"(""For more information, see Cognitive Services Compliance and Privacy notes."",)"


In [3]:
ALTER TABLE listings ADD COLUMN description_pii_safe text;
ALTER TABLE listings ADD COLUMN pii_entities azure_cognitive.entity[];

UPDATE listings
SET
    description_pii_safe = pii.redacted_text,
    pii_entities = pii.entities
FROM (SELECT id, description FROM listings WHERE description_pii_safe IS NULL OR pii_entities IS NULL ORDER BY id LIMIT 100) subset,
LATERAL azure_cognitive.recognize_pii_entities(subset.description, 'en-us') as pii
WHERE listings.id = subset.id;

SELECT pii_entities
FROM listings
WHERE entities IS NOT NULL
LIMIT 1;

UPDATE 50

(1 row(s) affected)

Total execution time: 00:00:08.280

pii_entities
"{""(craftsman,PersonType,\""\"",0.69)"",""(Zoka,Organization,\""\"",0.58)"",""(I-5,Address,\""\"",0.67)"",""(\""Rt 99\"",Address,\""\"",0.78)""}"
